<h1><strong>Extracción y tratamiento de datos</strong></h1>

El presente notebook busca tanto la extracción como el tratamiento de los datos a partir de los cuales se construirá el dataset. En primer lugar, se realizará una extracción de los datos a partir de su ubicación originale en la herramienta IBM Doors siguiento el formato DXL (DOORS eXtension Language), un lenguaje de scripting propio del fabricante para la escritura, lectura y tratamiento de datos.

En primer lugar realizaremos todas las importaciones de librerías necesarias para la ejecución del código:

In [18]:
!pip3 install import_ipynb
!pip3 install nlpaug
!pip3 install easy-nlp-augmentation
!pip3 install nltk
!pip3 install sentencepiece
!pip3 install sacremoses

import import_ipynb
import selenium_tests
import string
import nlpaug.augmenter.word as aumentador_palabras
import nlpaug.augmenter.sentence as aumentador_frases
from easy_text_augmenter import augment_random_character

import os
import re
import pickle
import numpy
import numpy as np
import yaml
from tqdm import tqdm
from screeninfo import get_monitors
import time
from getpass import getpass
import pandas as pd
import chardet
import subprocess
import random

import selenium.common.exceptions
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.relative_locator import RelativeBy
from selenium.webdriver.support import expected_conditions as EC

from huggingface_hub import snapshot_download
import nltk
import sentencepiece
from transformers import MarianMTModel, MarianTokenizer


[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Para la extracción de los datos y guardado en fichero CSV puede consultarse el código escrito en el fichero <strong>exportacion_TP_CSV.dxl</strong>. Tras una serie de preguntas al usuario para la obtención de los datos requeridos, se hará una llamada al script dxl mencionado para la exportación de datos a CSV. Todo ello puede verse en la siguiente celda:

In [ ]:
def obtener_cabecera_test(test):
    return test.split('\r\n')[0]

def generar_yaml(valor):
    if valor == 'Case':
        return 'URL: \r\n acciones: \r\n  - funcion:\r\n    params: '

    else:
        return ''

def obtener_titulos(tipos, tests):
    titulos = []

    i = 0
    j = 0
    while i < len(tipos):
        if tipos[i] == 'Test' or tipos[i] == '*':
            titulos.append(tests[i])
            j = i

        elif tipos[i] == 'Case':
            titulos.append(tests[j])

        i = i + 1

    return titulos

def obtener_dxl():
    directorio_onedrive = os.getenv('OneDrive')
    
    # Creamos el directorio que contendrá los CSV exportados
    directorio_exportacion = os.getcwd() + '\\ficheros_excel\\TPs\\'
    os.makedirs(directorio_exportacion, exist_ok=True)
    
    # Se pregunta al usuario los datos para poder acceder a los módulos correspondientes dentro de IBM Doors
    base_de_datos = input('Introduzca la base de datos a la que se ha de conectar: ')
    usuario = input('Introduzca su nombre de usuario: ')
    software = input('Introduzca el nombre del software cuyos casos de prueba desea extraer: ')
    nombre_proyecto = input('Introduzca el nombre del proyecto/versión: ')
    
    # Dado que no es posible pasar parámetros al script DXL o no se ha sabido cómo, se guardarán los datos introducidos por el usuario
    with open(directorio_onedrive + '\Documents\DOORS_y_CHANGE\SCRIPTS DXL\config_proyecto_sw.txt', mode='w') as f:
        f.write(software + '\n' + nombre_proyecto)
    
    # Hacemos la llamada al script exportacion_TP_CSV.dxl para la obtención de los ficheros que contienen las pruebas
    comando_a_ejecutar = r'"C:\Program Files\IBM\Rational\DOORS\9.7\bin\doors.exe" -d ' + base_de_datos + ' -user ' + usuario + ' -batch "' + directorio_onedrive + '\Documents\DOORS_y_CHANGE\SCRIPTS DXL\TP_CSV_EXPORT.dxl" -a "\\' + base_de_datos.split('@')[-1] + '\DXL\addins;\\' + base_de_datos.split('@')[-1] + '\DXL" -J \\' + base_de_datos.split('@')[-1] + '\DXL\project"'
    resultado = subprocess.call(comando_a_ejecutar)

    # Creamos el dataframe que contendrá todos los casos de prueba tras su lectura y procesamiento, así como las listas que contendrán las pruebas y los títulos
    df_resultante = pd.DataFrame(columns=['Tipo', 'Test', 'Titulo'])
    all_tests = []
    all_titles = []

    nombres_ficheros_csv = os.listdir(directorio_exportacion)

    # Eliminamos de la lista todo aquel fichero que pueda existir en el directorio que no sea CSV
    nombres_ficheros_csv = [nombre_fichero_csv for nombre_fichero_csv in nombres_ficheros_csv if nombre_fichero_csv.endswith('.csv')]

    for nombre_ficheros_csv in nombres_ficheros_csv:
        # Detectamos la codificación con que se guardaron, ya que los CSV contienen acentos y otros caracteres especiales
        with open(directorio_exportacion + nombre_ficheros_csv, mode='rb') as f:
            codificacion = chardet.detect(f.read())['encoding']

        nombre_fichero = directorio_exportacion + nombre_ficheros_csv
        with open(nombre_fichero, mode='r', encoding=codificacion) as f:
            print(time.strftime('%d/%m/%Y %H:%M:%S') + '\tProcesando ' + nombre_fichero + '...')
            contenido_fichero = f.read()
            contenido_fichero = contenido_fichero.split('\n')

            # Comprobamos si el caso de prueba aplica a la versión V2.01.13 o inferior. De no ser así, no exportará al CSV final
            i = 0
            while i < len(contenido_fichero):
                if 'Case' in contenido_fichero[i]:
                    j = i + 1
                    tested_versions = []
                    while j < len(contenido_fichero) and 'Case' not in contenido_fichero[j]:
                        if 'Execution' in contenido_fichero[j]:
                            tested_versions.append(contenido_fichero[j].split(';-;')[0])

                        j = j + 1

                    if '02.01.' not in ' '.join(tested_versions) and '02.00.' not in ' '.join(tested_versions):
                        contenido_fichero = contenido_fichero[0:i] + contenido_fichero[j:]

                i = i + 1

            # Eliminamos líneas que indican si la ejecución tuvo éxito o no
            contenido_fichero = [content for content in contenido_fichero if ';-;Execution;-;' not in content]

            # Eliminamos aquellas lineas referentes a tests sin casos de prueba
            i = 0
            while i < len(contenido_fichero):
                if i < len(contenido_fichero) - 1 and ';Test;' in contenido_fichero[i] and ';Test;' in contenido_fichero[i + 1]:
                    contenido_fichero.remove(contenido_fichero[i])

                else:
                    i = i + 1

            # Check if the next line is part of a case, in which occurrence we add a \n and write the value between double commas
            # Comprobamos si la siguiente línea es parte de un caso de prueba. De ser así, añadimos una nueva línea y escribimos el texto entre comillas dobles para que en Excel sea una única celda
            i = 2
            while i < len(contenido_fichero):
                if i < (len(contenido_fichero) - 1) and 'Case' not in contenido_fichero[i + 1] and 'Test' not in contenido_fichero[i + 1] and '*;' not in contenido_fichero[i + 1]:
                    contenido_fichero[i] = contenido_fichero[i] + '\r\n' + contenido_fichero[i + 1]
                    contenido_fichero.remove(contenido_fichero[i + 1])

                else:
                    i = i + 1

            # Separamos las columnas. La separación original era ;-; para que las , o ; que pudiesen haber no se tomasen como separación de columnas
            i = 0
            while i < len(contenido_fichero):
                contenido_fichero[i] = contenido_fichero[i].split(';-;')
                if len(contenido_fichero[i]) > 1:
                    contenido_fichero[i] = contenido_fichero[i][2:]

                i = i + 1

        # Introducimos el contenido en un dataframe
        df = pd.DataFrame(columns=contenido_fichero[0], data=contenido_fichero[1:])

        # Renombramos las columnas
        df.columns = ['Tipo', 'Test']

        # Añadimos la columna Titulo, que será la cabecera del objeto, es decir, el título del caso de prueba, que en DOORS se muestra en negrita
        df['Titulo'] = obtener_titulos(df.Tipo, df.Test)

        # Añadimos el dataframe al dataframe final, contenedor de todos los casos de prueba
        df_resultante = pd.concat([df_resultante, df], ignore_index=True)
        l = 0

    orden_nuevas_columnas = ['Tipo', 'Titulo', 'Test']
    df_resultante = df_resultante[orden_nuevas_columnas]

    # Hay casos de pruebas duplicados, donde una entrada hace referencia a una versión anterior a la que se está probando. Las buscamos y eliminamos
    df_resultante['Cabecera caso'] = df_resultante['Test'].apply(obtener_cabecera_test)

    # Comprobamos aquellos casos donde la cabecera es la misma a excepción de que una indica la versión y otra no
    cabeceras_sin_version = [cabecera for cabecera in df_resultante['Cabecera caso'] if '(V2' not in cabecera]

    for cabecera in cabeceras_sin_version:
        # Buscamos las filas que contienen cabecera, incuyendo la versión a partir de la cual aplica
        filas_encontradas = df_resultante.loc[(df_resultante['Test'].str.contains(cabecera + ' (V2', regex=False)) | (df_resultante['Cabecera caso'].str.contains(cabecera + '(V2', regex=False)) | (df_resultante['Cabecera caso'].str.contains(cabecera + '(v2', regex=False)) | (df_resultante['Cabecera caso'].str.contains(cabecera + '(v2', regex=False))]

        if len(filas_encontradas) > 0:
            # Obtenemos el contenido del test sin la cabecera, que se usará para comprobar que la única diferencia es la versión
            texto_test = '\r\n'.join(filas_encontradas.loc[filas_encontradas.index[0]]['Test'].split('\r\n')[1:])

            # Buscamos las filas que contienen la cabecera que estamos buscando, sin la versión, y obtenemos el índice
            filas_originales = df_resultante.loc[df_resultante['Test'].str.contains(cabecera + '\r\n', regex=False)]
            indice = filas_originales.index[0]

            # Si, en efecto, se trata de filas idénticas salvo la versión en la cabecera, borramos la entrada anterior
            if texto_test in filas_originales.loc[indice]['Test']:
                df_resultante.drop(index=indice, inplace=True)


    # Eliminamos la columna dado que ya no resulta necesaria
    df_resultante.drop(columns=['Cabecera caso'], inplace=True)

    # Reordenamos las columnas por el contenido de Test. De esa forma, se facilitará encontrar filas que puedan ser idénticas
    df_resultante.sort_values(by=['Titulo', 'Test'], inplace=True)

    # Ordenamos por el contenido en test
    df_resultante.reset_index(inplace=True, drop=True)

    # Añadimos la columna YAML con un contenido por defecto para facilitar la edición
    df_resultante['YAML'] = df_resultante['Tipo'].apply(generar_yaml)

    csv_resultante = directorio_exportacion + '..\\MODELO.csv'
    print(time.strftime('%d/%m/%Y %H:%M:%S') + '\tGuardando archivo ' + csv_resultante)

    intentar_guardar = True
    while intentar_guardar:
        try:
            df_resultante.to_csv(csv_resultante, index=None, sep=';', encoding='windows-1252')
            intentar_guardar = False

        except PermissionError:
            print('¡Error intentando guardar ' + csv_resultante + '! Cierre el fichero para poder guardarlo')
            time.sleep(15)

obtener_dxl()

En este momento se habrán exportado todos los casos de prueba y almacenado como CSV. Antes de continuar, se procederá a comprobar que los datos se extrajeron correctamente.

Tras dicha revisión, se procede a realizar un primer estudio de los datos, donde extraeremos la cantidad de palabras distintas y apariciones, cuántos casos de prueba tienen una longitud mayor de 512 caracteres y la longitud mayor encontrada entre los diversos casos de prueba. Todo ello será de utilidad para la elección del modelo pre-entrenado que mejores resultados pueda ofrecer.

In [4]:
def analizar_datos(nombre_fichero):
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\
    # Se abre el fichero para su análisis
    if nombre_fichero.upper().endswith('CSV'):
        df = pd.read_csv(directorio_ficheros_excel + nombre_fichero, sep=';', encoding='windows-1252')

    elif
        df = pd.read_excel(io=directorio_ficheros_excel + nombre_fichero, sheet_name=nombre_fichero.split('.')[0], index_col=None)
    
    else:
        print('No se ha podido abrir el fichero indicado para analizar datos.')
    
    # Definimos diccionarios, cuyas claves serán las diferentes palabras encontradas y su valor la cantidad de veces que se encontró
    conteo_palabras = {}
    conteo_palabras_minusculas = {}
    
    longitud_entrada_mas_larga = 0
    
    # Se extraen los valores de la columna de Test, pues contiene los textos a analizar
    valores = df['Test'].values.tolist()
    
    # Se introduce el conteo de palabras en el diccionario
    i = 0
    while i < len(valores):
        palabras = re.findall(r'(\b[a-zA-Z0-9]+\b)', valores[i])
        for palabra in palabras:
            try:
                conteo_palabras[palabra] = conteo_palabras[palabra] + 1
                conteo_palabras_minusculas[palabra.lower()] = conteo_palabras_minusculas[palabra.lower()] + 1
    
            except KeyError:
                conteo_palabras[palabra] = 1
                conteo_palabras_minusculas[palabra.lower()] = 1
    
        i = i + 1
    
    #valores = df['Test'].values.tolist()
    i = 0
    indice_entrada_mas_larga = 0
    num_entradas_mayores_512_palabras = 0
    while i < len(valores):
        entrada_texto = valores[i].split()
    
        # Eliminamos signos de puntuación dado que no cuentan como tokens
        while '.' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index('.')])
    
        while ',' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index(',')])
    
        while ';' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index(';')])
    
        while '_' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index('_')])
    
        while '-' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index('-')])
    
        while '"' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index('"')])
    
        while '\'' in entrada_texto:
            entrada_texto.remove(entrada_texto[entrada_texto.index('\'')])
    
        if len(entrada_texto) > longitud_entrada_mas_larga:
            longitud_entrada_mas_larga = len(valores[i].split())
            indice_entrada_mas_larga = i
    
        if len(entrada_texto) > 512:
            num_entradas_mayores_512_palabras = num_entradas_mayores_512_palabras + 1
    
        i = i + 1
    
    print('Cantidad total de entradas de texto, incluyendo títulos:', len(valores))
    print('\nPalabras diferentes: ' + str(len(conteo_palabras.keys())))
    print('\nPalabras distintas, con todas las letras en minúsculas: ' + str(len(conteo_palabras_minusculas.keys())))
    print('\nEntradas con longitud superior a 512 palabras:', num_entradas_mayores_512_palabras,
          '\nMayor longitud (en palabras o tokens) encontrada para texto de entrada:', longitud_entrada_mas_larga, '. Índice: ',
          indice_entrada_mas_larga)
    
    with open(os.getcwd() + '\\ficheros_excel\\conteo_datos.csv', mode='w', encoding='windows-1252') as f:
        f.write('Palabra;Veces escrita\n')
        for key_palabra in conteo_palabras.keys():
            f.write(key_palabra + ';' + str(conteo_palabras[key_palabra]) + '\n')
    
    with open(os.getcwd() + '\\ficheros_excel\\conteo_datos_minúsculas.csv', mode='w', encoding='windows-1252') as f:
        f.write('Palabra;Veces escrita\n')
        for key_palabra in conteo_palabras_minusculas.keys():
            f.write(key_palabra + ';' + str(conteo_palabras_minusculas[key_palabra]) + '\n')

analizar_datos('MODELO.csv')

Cantidad total de entradas de texto, incluyendo títulos: 1039

Palabras diferentes: 2882

Palabras distintas, con todas las letras en minúsculas: 2356

Entradas con longitud superior a 512 palabras: 0 
Mayor longitud encontrada para texto de entrada: 441 . Índice:  812


Tras el primer análisis de los datos, procederemos a analizar la página web sobre la que se realizarán pruebas. Para ello, codificaremos funciones que ayudarán en la extracción de xpaths y capturas de imagen asociadas, tanto de forma voraz donde se intentará de forma automática realizar clics y clics derecho de ratón o ponerlo sobre elementos para así obtener el mayor contenido posible como manualmente, donde se indicará el xpath a partir del cual obtener la información y que podrá servir para completar la obtención de todos aquellos elementos que puedan existir.

Antes de dichas funciones, se codifica aquella que se encargará de guardar los resultados obtenidos en la obtención de los diversos xpath

In [19]:
def guardar_df_como_excel(df, nombre_fichero, aplicar_filtro=True, aplicar_colorizacion=True, contiene_imagenes=False):
    intentar_guardar = True

    # Creamos una lista vacía que contendrá los DataFrames, pudiendo ser uno o varios en función del número de filas que tenga
    # El motivo de esta acción es, además de evitar que supere el tamaño máximo permitido, poder abrir el fichero en excel
    dfs = []
    if len(df) > 10000:
        i = 0
        while i < len(df):
            if i + 10000 < len(df):
                dfs.append(df[i:i + 10000])

            else:
                dfs.append(df[i:])

            i = i + 10000

    else:
        dfs.append(df)

    i = 0
    while i < len(dfs):
        while intentar_guardar:
            try:
                if len(dfs) > 1:
                    excel_writer = pd.ExcelWriter(nombre_fichero.replace('.xlsx', '_' + str(i) + '.xlsx'))

                else:
                    excel_writer = pd.ExcelWriter(nombre_fichero)

                nombre_hoja = nombre_fichero.replace('.xlsx', '')
                if '\\' in nombre_hoja:
                    nombre_hoja = nombre_hoja.split('\\')[-1]

                dfs[i].to_excel(excel_writer=excel_writer, sheet_name=nombre_hoja, index=None)
                workbook = excel_writer.book
                worksheet = excel_writer.sheets[nombre_hoja]

                # Calculamos el ancho de la columna, que ayudará a poder visualizar el contenido sin tener que modificar en excel
                columnas_hoja = dfs[i].columns.tolist()
                for nombre_columna in columnas_hoja:
                    ancho_columna = len(nombre_columna)
                    valores_columna = dfs[i][nombre_columna]

                    for texto_celda in valores_columna:
                        if type(texto_celda) is str and '\r\n' in texto_celda:
                            texto_celda = texto_celda.split('\r\n')
                            long_max_valores_columna = len(max(texto_celda, key=len))

                        elif type(texto_celda) is str and '\n' in texto_celda:
                            texto_celda = texto_celda.split('\n')
                            long_max_valores_columna = len(max(texto_celda, key=len))

                        elif type(texto_celda) is str:
                            long_max_valores_columna = len(texto_celda)

                        if long_max_valores_columna > ancho_columna:
                            ancho_columna = long_max_valores_columna

                        # En caso de que se llegue a una columna a una longitud superior a 150, establecemos la anchura máxima a 150 y pasamos a procesar la siguiente columna
                        if ancho_columna > 150:
                            ancho_columna = 150
                            break

                    longitud_columna = ancho_columna + 6
                    posicion_columna = columnas_hoja.index(nombre_columna)
                    worksheet.set_column(posicion_columna, posicion_columna, longitud_columna)

                # Colorear celdas de filas impares si se indica por parámetro. Puede ser útil para mejorar visualización del fichero excel
                if aplicar_colorizacion:
                    formato_celda = workbook.add_format()
                    formato_celda.set_bg_color('c5d9f1')

                    j = 1
                    rows_indexes = df.index.tolist()
                    while j < len(df):
                        if j % 2 != 0:
                            k = 0
                            while k < len(columnas_hoja):
                                value_to_modify = df.loc[rows_indexes[j]][columnas_hoja[k]]
                                worksheet.write(j + 1, k, value_to_modify, formato_celda)
                                k = k + 1

                        j = j + 1

                # Aplicar filtros si se indica, equivalente a ir a Datos -> Filtro en Excel
                if aplicar_filtro:
                    worksheet.autofilter(0, 0, len(df), len(columnas_hoja) - 1)

                # Sustituimos el nombre del fichero en la columna PNG por el contenido de la imagen en la propia celda
                if contiene_imagenes:
                    fila = 1
                    while fila <= len(df):
                        fichero_imagen = df.loc[fila-1]['PNG']
                        if fichero_imagen is not None and fichero_imagen != '':
                            try:
                                worksheet.embed_image(fila, df.columns.tolist().index('PNG'), fichero_imagen)

                            except (FileNotFoundError, Exception) as e:
                                print(e)
                                pass

                        fila = fila + 1

                # Terminamos de guardar el fichero. Si fuese bien, se establece intentar guardar a False para salir del bucle
                excel_writer.close()
                intentar_guardar = False

            except PermissionError:
                print('¡Error, parece que el fichero ' + nombre_fichero + ' se encuentra abierto! Cierre el fichero para poder guardarlo')
                time.sleep(10)

        i = i + 1

Por otra parte, se muestra a continuación el código utilizado para el acceso al software, inicio y cierre de sesión para poder posteriormente extraer información:

In [ ]:
from screeninfo import get_monitors

# Dado que el Software implementado está diseñado para pantallas de 4K, obtenemos con ayuda de esta función las pantallas disponibles y su resolución
def obtener_info_pantallas():
    monitores = get_monitors()

    propiedades_monitor = None
    ancho_monitor = 0
    alto_monitor = 0
    i = 0
    while i < len(monitores):
        if monitores[i].width > ancho_monitor and monitores[i].height > alto_monitor:
            propiedades_monitor = monitores[i]

        i = i + 1

    return propiedades_monitor

# Obtenemos las propiedades del monitor y creamos un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
propiedades_monitor = obtener_info_pantallas()
tests = selenium_tests.Tests(propiedades_monitor)

# Configuramos el driver
tests.configurar_driver()

# Iniciamos la sesión
tests.iniciar_sesion(params={})

Los siguientes métodos serán los utilizados para la extracción de contenido de la página web sobre la que se realizarán las pruebas

In [27]:
def obtener_df_xpaths():
    df_xpaths = pd.DataFrame(columns=['XPATH', 'PNG', 'ACCESO DESDE', 'PALABRAS CLAVE', 'CONTENIDO HTML', 'XPATHS_CONTENIDO_HTML', 'REVISADO'])
    datos_elementos_no_repetibles = None

    # Comprobamos si ya existen datos almacenados para no empezar desde cero innecesariamente
    try:
        with open('df_xpaths.pickle', mode='rb') as fichero_pickle:
            df_xpaths = pickle.load(fichero_pickle)

            # Creamos unas listas de elementos cuyos valores y etiquetas o xpaths son cambiantes y bastaría con tener uno a modo de ejemplo
            datos_elementos_no_repetibles = ['class="app"', 'hist-cell TRANSPARENT', 'Total flights', 'hist-cell ifr-arr', 'A2SA', 'A1SR4', 'GCCC', 'PREOPS', 'nmOTMVPeriodPeak', 'timeline', 'rowFL']

            for dato_no_repetible in datos_elementos_no_repetibles:
                existe = df_xpaths['XPATH'].str.contains(dato_no_repetible)
                if existe.any() is False:
                    datos_elementos_no_repetibles.remove(dato_no_repetible)

    # Si se produce un error, se debe a que no existe el fichero. Lo capturamos y no hacemos nada
    except FileNotFoundError:
        pass

    return df_xpaths

def obtener_xpath_de_html(contenido_html):
    contenido_html = contenido_html.split('<')
    xpath_a_devolver = '//' + re.sub(r'data.+""\s', '', contenido_html[1][:contenido_html[1].index('>')])

    xpaths_especiales = ['svg', 'path']
    if xpath_a_devolver.split()[0].replace('//', '') in xpaths_especiales:
        xpath_a_devolver = xpath_a_devolver.replace(xpath_a_devolver.split()[0], '//*[name()="' + xpath_a_devolver.split()[0].replace('//', '') + '"]')

    xpath_a_devolver = re.sub(r'"\s', '"][@', xpath_a_devolver)
    xpath_a_devolver = xpath_a_devolver.replace(' ', '[@', 1)
    if '[' in xpath_a_devolver:
        xpath_a_devolver = xpath_a_devolver + ']'

    try:
        texto_elemento = contenido_html[1]
        texto_elemento = texto_elemento[texto_elemento.index('>') + 1:]
        texto_elemento = re.sub(r'^\s{1,}|\s{1,}$', '', texto_elemento)
        if len(texto_elemento) > 0 and re.search(r'(text\(\)\,\s"\d+:\d+)', xpath_a_devolver) is None and re.search(r'(text\(\),\s"\d+\/\d+\/\d+)', xpath_a_devolver) is None:
            xpath_a_devolver = xpath_a_devolver + '[contains(text(), "' + texto_elemento + '")]'

    except (IndexError, ValueError):
        # Se da el error si es un único tag sin otro a continuación o sin texto: No necesitamos hacer nada, por lo que puede ignorarse
        pass

    return xpath_a_devolver


def obtener_xpaths_de_html(contenido_html):
    posibles_xpaths = []

    contenido_html_en_lista = contenido_html.split('><')
    contenido_html_en_lista = [('<' + contenido) if contenido.startswith('<') is False else contenido for contenido in contenido_html_en_lista]
    contenido_html_en_lista = [(contenido + '>') if contenido.endswith('>') is False else contenido for contenido in contenido_html_en_lista]

    # Generamos en primer lugar un xpath a partir de cada una de las etiquetas
    i = 0
    while i < len(contenido_html_en_lista):
        if contenido_html_en_lista[i].startswith('<!') is False:
            posibles_xpaths.append(obtener_xpath_de_html(contenido_html_en_lista[i]))

        i = i + 1

    l = 0
    return posibles_xpaths


def obtener_xpath_de_elemento(elemento):
    excepciones_nombre_tag = ['svg', 'path']
    if elemento.tag_name not in excepciones_nombre_tag:
        xpath_a_devolver = '//' + elemento.tag_name

    else:
        xpath_a_devolver = '//*[name()="' + elemento.tag_name + '"]'

    atributos = tests.driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', elemento)
    if atributos is not None and len(atributos.keys()) > 0:
        # Comprobamos en primer lugar si el elemento tuviese un id, en cuyo caso se toma solo ese atributo dado que es un valor único
        if 'id' in atributos:
            xpath_a_devolver = xpath_a_devolver + '[@id="' + elemento.get_attribute('id') + '"]'

        else:
            for atributo in atributos:
                if elemento.get_attribute(atributo) is not None and elemento.get_attribute(atributo) != '':
                    xpath_a_devolver = xpath_a_devolver + '[@' + atributo + '="' + elemento.get_attribute(atributo) + '"]'

    try:
        outer_html = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""', '', elemento.get_attribute('outerHTML'))
        texto_elemento = outer_html.split('<')[1]
        texto_elemento = texto_elemento[texto_elemento.index('>') + 1:]
        texto_elemento = re.sub(r'^\s{1,}|\s{1,}$', '', texto_elemento)
        if 'Last Update' in elemento.text and '\n' not in elemento.text:
            l = 0

        if len(texto_elemento) > 0 and re.search(r'(text\(\)\,\s"\d+:\d+)', xpath_a_devolver) is None and re.search(r'(text\(\),\s"\d+\/\d+\/\d+)', xpath_a_devolver) is None:
            xpath_a_devolver = xpath_a_devolver + '[contains(text(), "' + texto_elemento + '")]'

    except (IndexError, ValueError):
        # Se da el error si es un único tag sin otro a continuación: No necesitamos hacer nada, por lo que puede ignorarse
        pass

    return xpath_a_devolver


def obtener_xpaths_voraz():
    datos_elementos_no_repetibles = None

    # Obtenemos el dataframe con los xpaths, que estará vacío en caso de ser la primera ejecución
    df_xpaths = obtener_df_xpaths()

    # Creamos el directorio que contendrá las capturas. De existir el directorio no se hace nada más
    os.makedirs(os.getcwd() + '\\capturas', exist_ok=True)

    # Se espera a detectar que existe el contenedor cuya clase contiene el texto tab-ws
    params = {'esperar': {'xpath': '//div[contains(@class,"tab-ws")]'}, 'tipo_espera': 'presencia'}
    tests.esperar_elemento(params)
    time.sleep(5)

    # Preguntamos al usuario la URL de acceso
    url_acceso = input('Indique la URL a la que se debe acceder:' )
    tests.driver.get(url_acceso)

    # En caso de estar actualizándose los datos, esperamos a que finalice
    params = {'esperar': {'xpath': '//div[@class="v-card__text"][contains(text(), "Please stand by")]'}, 'tipo_espera': 'invisibilidad'}
    tests.esperar_elemento(params)

    # Guardamos el contenido HTML, que nos servirá posteriormente para detectar otros posibles xpaths
    # Sustituimos valores de fecha y hora, ya que pueda hacer que parezca que dos etiquetas son distintas cuando la diferencia es la hora actial
    contenido_html = re.sub(r'\d+:\d+|\d+\/\d+\/\d+', '', tests.driver.page_source)
    contenido_html = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', contenido_html)

    try:
        if len(df_xpaths) == 0:
            # Buscamos todos los elementos que haya en la página
            elementos_encontrados = tests.driver.find_elements(By.XPATH, '//body//*')

            tags_a_eliminar = ['html', 'link', 'title', 'style', 'script', 'head', 'meta']
            elementos_encontrados = [elemento for elemento in elementos_encontrados if elemento is not None and elemento.tag_name is not None and elemento.tag_name not in tags_a_eliminar]

            for elemento in tqdm(elementos_encontrados, unit='xpath', total=len(elementos_encontrados)):
                try:
                    xpath_a_aniadir = obtener_xpath_de_elemento(elemento)

                    if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():
                        try:
                            contenido_html_sin_separar = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', elemento.get_attribute('outerHTML'))
                            xpaths_contenido_html = obtener_xpaths_de_html(contenido_html_sin_separar)

                            # Obtenemos las palabras clave, que serán de utilidad para poder buscar los xpaths necesarios
                            palabras_clave = re.findall(r'\b\w+\b', xpath_a_aniadir)
                            palabras_clave.append(re.sub(r'\d+\/\d+\/\d+|\d{2}:\d{2}', '', elemento.text).replace('\n\n', ',').replace('\n', ','))

                            j = 10
                            while j < len(palabras_clave):
                                palabras_clave.insert(j, '\n')

                                j = j + 10

                            # Eliminamos los valores repetidos, ya que en este caso no es necesario tener valores duplicados
                            palabras_clave = list(set(palabras_clave))
                            palabras_clave = ','.join(palabras_clave)

                            # Guardamos la imagen en un fichero, cuyo nombre será el del índice si se hizo captura
                            if type(elemento.screenshot_as_png) is bytes:
                                nombre_fichero_imagen = os.getcwd() + '\\capturas\\captura' + str(len(df_xpaths)) + '.png'
                                elemento.screenshot(nombre_fichero_imagen)
                                df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, nombre_fichero_imagen, '', palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                            else:
                                df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', '',  palabras_clave, contenido_html_sin_separar,  '\n'.join(xpaths_contenido_html), '']

                        except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                            df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', '',  palabras_clave, contenido_html_sin_separar, '', '']

                except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                    pass

        # Convertimos contenido_html a lista, pues posteriormente se utilizará para comprobar si acciones como un clic hace que se muestre nuevo contenido
        contenido_html = contenido_html.split('><')
        contenido_html = [('<' + contenido) if contenido.startswith('<') is False else contenido for contenido in contenido_html]
        contenido_html = [(contenido + '>') if contenido.endswith('>') is False else contenido for contenido in contenido_html]

        acciones = ['clic_derecho', 'hover', 'clic']

        # Ya obtenidos los xpath de la web, ejecutamos las acciones de clic, clic derecho y hover y buscamos posibles xpaths a añadir
        barra_progreso = tqdm(total=len(df_xpaths), unit='xpath(s)')
        i = 0
        while i < len(df_xpaths):
            if df_xpaths.loc[i]['REVISADO'] != 'SI':
                for accion in acciones:
                    if accion == acciones[-1]:
                        # Lo marcamos como revisado para, en caso de retomar el proceso, no volver a hacer lo ya realizado
                        df_xpaths.loc[i]['REVISADO'] = 'SI'

                    try:
                        # En caso de estar actualizándose los datos, esperamos a que finalice
                        params = {'esperar': {'xpath': '//div[@class="v-card__text"][contains(text(), "Please stand by")]'}, 'tipo_espera': 'invisibilidad'}
                        tests.esperar_elemento(params)

                        match accion:
                            case 'clic':
                                # Hacemos clic sobre el elemento
                                tests.hacer_clic({'xpath': df_xpaths.loc[i]['XPATH']})

                            case 'clic_derecho':
                                # Hacemos clic derecho sobre el elemento
                                tests.hacer_clic_derecho({'xpath': df_xpaths.loc[i]['XPATH']})

                            case 'hover':
                                 # Intentamos hacer hover sobre el elemento
                                 tests.hacer_hover({'xpath': df_xpaths.loc[i]['XPATH']})

                        # Obtenemos el nuevo contenido HTML eliminando datos de fecha y/o hora y comparamos con el almacenado para ver si es diferente
                        # En primer lugar, eliminamos etiquetas de comentarios y datos de fecha y hora que puedan hacer que hay diferencias que para análisis no deberían existir en realidad
                        nuevo_contenido_html = re.sub(r'\d+:\d+|\d+\/\d+\/\d+', '', tests.driver.page_source).replace('<!---->', '')
                        nuevo_contenido_html = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', nuevo_contenido_html)
                        nuevo_contenido_html = nuevo_contenido_html.split('><')
                        nuevo_contenido_html = [('<' + contenido) if contenido.startswith('<') is False else contenido for contenido in nuevo_contenido_html]
                        nuevo_contenido_html = [(contenido + '>') if contenido.endswith('>') is False else contenido for contenido in nuevo_contenido_html]

                        # Eliminamos aquello que no sea diferente o nuevo
                        nuevo_contenido_html = [contenido for contenido in nuevo_contenido_html if contenido not in contenido_html]

                        # Eliminamos aquellos elementos que puedan considerarse repetidos
                        j = 0
                        while j < len(datos_elementos_no_repetibles):
                            k = 0
                            while k < len(nuevo_contenido_html):
                                if nuevo_contenido_html[k] in datos_elementos_no_repetibles[j]:
                                    datos_elementos_no_repetibles.remove(datos_elementos_no_repetibles[j])

                                else:
                                    k = k + 1

                            j = j + 1

                        for contenido in nuevo_contenido_html:
                            xpath_a_aniadir = obtener_xpath_de_html(contenido)

                            if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():
                                try:
                                    # Buscamos todos los elementos que haya en la página
                                    elementos_encontrados = tests.driver.find_elements(By.XPATH, xpath_a_aniadir + '//*')
                                    elementos_encontrados = [elemento for elemento in elementos_encontrados if elemento.tag_name not in tags_a_eliminar]

                                    for elemento in elementos_encontrados:
                                        try:
                                            xpath_a_aniadir = obtener_xpath_de_elemento(elemento)

                                            if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():# and '[' in xpath_a_aniadir:
                                                try:
                                                    contenido_html_sin_separar = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', elemento.get_attribute('outerHTML'))
                                                    xpaths_contenido_html = obtener_xpaths_de_html(contenido_html_sin_separar)

                                                    # Obtenemos las palabras clave, que serán de utilidad para poder buscar los xpaths necesarios
                                                    palabras_clave = re.findall(r'\b\w+\b', xpath_a_aniadir)
                                                    palabras_clave.append(re.sub(r'\d+\/\d+\/\d+|\d{2}:\d{2}', '', elemento.text).replace('\n\n', ',').replace('\n', ','))

                                                    j = 10
                                                    while j < len(palabras_clave):
                                                        palabras_clave.insert(j, '\n')

                                                        j = j + 10

                                                    # Eliminamos los valores repetidos, ya que en este caso no es necesario tener valores duplicados
                                                    palabras_clave = list(set(palabras_clave))
                                                    palabras_clave = ','.join(palabras_clave)

                                                    # Si el botón de Cancelar, Cerrar o No está presente y estamos en la acción de clic, haremos clic sobre él para poder continuar
                                                    if accion == 'clic' and ('CANCEL' in xpath_a_aniadir.upper()) or ('CLOSE' in xpath_a_aniadir.upper()) or ('NO' in xpath_a_aniadir.upper()):
                                                        tests.hacer_clic({'xpath': xpath_a_aniadir})

                                                    # Guardamos la imagen en un fichero, cuyo nombre será el del índice si se hizo captura
                                                    if type(elemento.screenshot_as_png) is bytes:
                                                        nombre_fichero_imagen = os.getcwd() + '\\capturas\\captura' + str(len(df_xpaths)) + '.png'
                                                        df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, nombre_fichero_imagen, df_xpaths.loc[i]['XPATH'], palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                                                        # Guardamos la imagen en un fichero, cuyo nombre será el del índice
                                                        elemento.screenshot(nombre_fichero_imagen)

                                                except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                                                    df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', df_xpaths.loc[i]['XPATH'], palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                                        except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException, selenium.common.exceptions.ElementClickInterceptedException):
                                            pass

                                except (selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.ElementClickInterceptedException):
                                    pass

                    except (AttributeError, selenium.common.exceptions.ElementNotInteractableException,selenium.common.exceptions.ElementClickInterceptedException):
                        pass

                    # En caso de que suceda algún error no contemplado, lo capturamos para poder guardar todo lo que se haya encontrado
                    except Exception:
                        # Guardamos fichero pickle para, en caso de volver a ejecutar, poder hacerlo desde donde nos hubiésemos quedado
                        with open('df_xpaths.pickle', mode='wb') as fichero_pickle:
                            pickle.dump(df_xpaths, fichero_pickle)

                        # Acciones como CTRL+C provocan este error. Lo capturamos para poder guardar lo hecho y por si se quiere revisar durante el proceso que va por buen camino
                        print('Se ha detectado la ejecución de CTRL+C. Guardando hasta lo obtenido')
                        guardar_df_como_excel(df_xpaths, os.getcwd() + '\\ficheros_excel\\xpaths.xlsx', contiene_imagenes=True)

            i = i + 1
            barra_progreso.total = len(df_xpaths)
            barra_progreso.update(1)

        barra_progreso.close()

        # Guardamos el fichero Excel
        guardar_df_como_excel(df_xpaths, os.getcwd() + '\\ficheros_excel\\xpaths.xlsx', contiene_imagenes=True)

    except KeyboardInterrupt:
        # Guardamos fichero pickle para, en caso de volver a ejecutar, poder hacerlo desde donde nos hubiésemos quedado
        with open('df_xpaths.pickle', mode='wb') as fichero_pickle:
            pickle.dump(df_xpaths, fichero_pickle)

        # Acciones como CTRL+C provocan este error. Lo capturamos para poder guardar lo hecho y por si se quiere revisar durante el proceso que va por buen camino
        print('Se ha detectado la ejecución de CTRL+C. Guardando hasta lo obtenido')
        guardar_df_como_excel(df_xpaths, os.getcwd() + '\\ficheros_excel\\xpaths.xlsx', contiene_imagenes=True)


def obtener_xpaths_asistido():
    datos_elementos_no_repetibles = None

    # Obtenemos el dataframe con los xpaths, que estará vacío en caso de ser la primera ejecución
    df_xpaths = obtener_df_xpaths()

    # Creamos el directorio que contendrá las capturas. De ya existir, no se hace nada
    os.makedirs(os.getcwd() + '\\capturas', exist_ok=True)

    params = {'esperar': {'xpath': '//div[contains(@class,"tab-ws")]'}, 'tipo_espera': 'presencia'}
    tests.esperar_elemento(params)
    time.sleep(5)

    # Preguntamos al usuario la URL de acceso
    url_acceso = input('Indique la URL a la que se debe acceder:' )
    tests.driver.get(url_acceso)

    # En caso de estar actualizándose los datos, esperamos a que finalice
    params = {'esperar': {'xpath': '//div[@class="v-card__text"][contains(text(), "Please stand by")]'}, 'tipo_espera': 'invisibilidad'}
    tests.esperar_elemento(params)

    # Guardamos el contenido HTML, que nos servirá posteriormente para detectar otros posibles xpaths
    # Sustituimos valores de fecha y hora, ya que podría hacer que parezca que dos etiquetas son distintas cuando la diferencia es la hora actual
    contenido_html = re.sub(r'\d+:\d+|\d+\/\d+\/\d+', '', tests.driver.page_source)
    contenido_html = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', contenido_html)

    salir = False
    while salir is False:
        try:
            # Preguntamos al usuario por el xpath a buscar
            xpath_a_buscar = input('Introduzca el xpath a buscar, o escriba salir para finalizar: ')
            
            if 'salir' in xpath_a_buscar:
                salir = True
                break
            
            # Si el xpath indicado no acaba en //*, se lo añadimos para que busque el contenido dentro del tag
            if xpath_a_buscar.endswith('//*') is False:
                xpath_a_buscar = xpath_a_buscar + '//*'
            
            # Buscamos todos los elementos que haya en la página a partir del xpath dado
            elementos_encontrados = tests.driver.find_elements(By.XPATH, xpath_a_buscar)

            tags_a_eliminar = ['html', 'link', 'title', 'style', 'script', 'head', 'meta']
            elementos_encontrados = [elemento for elemento in elementos_encontrados if elemento.tag_name not in tags_a_eliminar]

            for elemento in tqdm(elementos_encontrados, unit='xpath', total=len(elementos_encontrados)):
                try:
                    xpath_a_aniadir = obtener_xpath_de_elemento(elemento)

                    if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():
                        try:
                            contenido_html_sin_separar = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', elemento.get_attribute('outerHTML'))
                            xpaths_contenido_html = obtener_xpaths_de_html(contenido_html_sin_separar)

                            # Obtenemos las palabras clave, que serán de utilidad para poder buscar los xpaths necesarios
                            palabras_clave = re.findall(r'\b\w+\b', xpath_a_aniadir)
                            palabras_clave.append(re.sub(r'\d+\/\d+\/\d+|\d{2}:\d{2}', '', elemento.text).replace('\n\n', ',').replace('\n', ','))

                            j = 10
                            while j < len(palabras_clave):
                                palabras_clave.insert(j, '\n')

                                j = j + 10

                            # Eliminamos los valores repetidos, ya que en este caso no es necesario tener valores duplicados
                            palabras_clave = list(set(palabras_clave))
                            palabras_clave = ','.join(palabras_clave)

                            # Guardamos la imagen en un fichero, cuyo nombre será el del índice si se hizo captura
                            if type(elemento.screenshot_as_png) is bytes:
                                nombre_fichero_imagen = os.getcwd() + '\\capturas\\captura' + str(len(df_xpaths)) + '.png'
                                elemento.screenshot(nombre_fichero_imagen)
                                df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, nombre_fichero_imagen, xpath_a_buscar, palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                            else:
                                df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', xpath_a_buscar,  palabras_clave, contenido_html_sin_separar,  '\n'.join(xpaths_contenido_html), '']

                        except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                            df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', xpath_a_buscar,  palabras_clave, contenido_html_sin_separar, '', '']

                except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                    pass

            # Convertimos contenido_html a lista, pues posteriormente se utilizará para comprobar si acciones como un clic hace que se muestre nuevo contenido
            contenido_html = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', elementos_encontrados[0].get_attribute('outerHTML'))
            contenido_html = contenido_html.split('><')
            contenido_html = [('<' + contenido) if contenido.startswith('<') is False else contenido for contenido in contenido_html]
            contenido_html = [(contenido + '>') if contenido.endswith('>') is False else contenido for contenido in contenido_html]

            # Eliminamos aquellos elementos que puedan considerarse repetidos
            j = 0
            while j < len(datos_elementos_no_repetibles):
                k = 0
                while k < len(contenido_html):
                    if contenido_html[k] in datos_elementos_no_repetibles[j]:
                        datos_elementos_no_repetibles.remove(datos_elementos_no_repetibles[j])

                    else:
                        k = k + 1

                j = j + 1

            for contenido in contenido_html:
                xpath_a_aniadir = obtener_xpath_de_html(contenido)

                if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():
                    try:
                        # Buscamos todos los elementos que haya en la página
                        elementos_encontrados = tests.driver.find_elements(By.XPATH, xpath_a_aniadir + '//*')
                        elementos_encontrados = [elemento for elemento in elementos_encontrados if elemento.tag_name not in tags_a_eliminar]

                        for elemento in elementos_encontrados:
                            try:
                                xpath_a_aniadir = obtener_xpath_de_elemento(elemento)

                                if xpath_a_aniadir not in df_xpaths['XPATH'].tolist():# and '[' in xpath_a_aniadir:
                                    try:
                                        contenido_html_sin_separar = re.sub(r'\sdata[a-zA-Z0-9\-]{1,}=""|<!---->', '', elemento.get_attribute('outerHTML'))
                                        xpaths_contenido_html = obtener_xpaths_de_html(contenido_html_sin_separar)

                                        # Obtenemos las palabras clave, que serán de utilidad para poder buscar los xpaths necesarios
                                        palabras_clave = re.findall(r'\b\w+\b', xpath_a_aniadir)
                                        palabras_clave.append(re.sub(r'\d+\/\d+\/\d+|\d{2}:\d{2}', '', elemento.text).replace('\n\n', ',').replace('\n', ','))

                                        j = 10
                                        while j < len(palabras_clave):
                                            palabras_clave.insert(j, '\n')

                                            j = j + 10

                                        # Eliminamos los valores repetidos, ya que en este caso no es necesario tener valores duplicados
                                        palabras_clave = list(set(palabras_clave))
                                        palabras_clave = ','.join(palabras_clave)

                                        # Guardamos la imagen en un fichero, cuyo nombre será el del índice si se hizo captura
                                        if type(elemento.screenshot_as_png) is bytes:
                                            nombre_fichero_imagen = os.getcwd() + '\\capturas\\captura' + str(len(df_xpaths)) + '.png'
                                            df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, nombre_fichero_imagen, '', palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                                            # Guardamos la imagen en un fichero, cuyo nombre será el del índice
                                            elemento.screenshot(nombre_fichero_imagen)

                                    except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException):
                                        df_xpaths.loc[len(df_xpaths)] = [xpath_a_aniadir, '', '', palabras_clave, contenido_html_sin_separar, '\n'.join(xpaths_contenido_html), '']

                            except (selenium.common.exceptions.WebDriverException, selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.StaleElementReferenceException, selenium.common.exceptions.ElementClickInterceptedException):
                                pass

                    except (selenium.common.exceptions.NoSuchElementException, selenium.common.exceptions.InvalidSelectorException, selenium.common.exceptions.ElementClickInterceptedException):
                        pass

            # Guardamos el fichero Excel
            guardar_df_como_excel(df_xpaths, os.getcwd() + '\\ficheros_excel\\xpaths.xlsx', contiene_imagenes=True)

        except KeyboardInterrupt:
            # Guardamos fichero pickle para, en caso de volver a ejecutar, poder hacerlo desde donde nos hubiésemos quedado
            with open('df_xpaths.pickle', mode='wb') as fichero_pickle:
                pickle.dump(df_xpaths, fichero_pickle)

            # Acciones como CTRL+C provocan este error. Se captura para poder guardar la información obtenida
            print('Se ha detectado la ejecución de CTRL+C. Guardando hasta lo obtenido')
            guardar_df_como_excel(df_xpaths, os.getcwd() + '\\ficheros_excel\\xpaths.xlsx', contiene_imagenes=True)

En la siguiente celda se podrá elegir el método de obtención de xpaths deseados. Una vez elegido, se procederá a la extracción hasta su finalización, ya sea por haber terminado el proceso en el caso del método voraz o por finalización por parte del usuario para cualquiera de los métodos de extracción de información:

In [ ]:
tipo_obtencion = input('Indique el tipo de obtención de XPATHS deseado:\n1) Voraz\n2) Asistido\n Opción: ')

match tipo_obtencion:
    case '1':
        obtener_xpaths_voraz()

    case '2':
        obtener_xpaths_asistido()

# Cerramos la sesión
tests.cerrar_sesion()

A continuación se abrirá el CSV que contiene todos los casos de prueba, sobre el que se añadirán las columnas de posibles índices de datos y sugerencias. Esto busca la facilitación de la redacción de las clases o valores de salida, es decir, la creación del YAML correspondiente a cada caso de prueba.

In [31]:
def obtener_posiciones_datos_y_sugerencias():
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
    
    # Abrimos el fichero Excel que contendrá los xpaths asociados, pues a través de la columna de palabras clave comprobaremos si se puede corresponder a un caso de prueba
    df_xpaths = pd.read_excel(io=directorio_ficheros_excel + 'xpaths.xlsx', sheet_name='xpaths', index_col=None)

    # Detectamos la codificación del fichero, para así evitar problemas de acentos y otros caracteres especiales
    codificacion_detectada = None
    with open(directorio_ficheros_excel + 'MODELO.csv', mode='rb') as f:
        contenido_fichero = f.read()
        codificacion_detectada = chardet.detect(contenido_fichero)['encoding']

    f.close()

    # Una vez detectada, abrimos el CSV que contiene nuestro modelo
    df_modelo = pd.read_csv(directorio_ficheros_excel + 'MODELO.csv', sep=';', encoding=codificacion_detectada, index_col=None)
    
    # Añadimos una columna en el DataFrame del modelo que indicará posibles filas en que se encuentren el/los xpaths que buscamos para el caso de prueba
    # Añadiremos además otra con sugerencias
    df_modelo['POSIBLES_INDICES_DATOS'] = [''] * len(df_modelo)
    df_modelo['SUGERENCIAS'] = [''] * len(df_modelo)

    # Creamos una lista que contiene palabras a eliminar del listado de palabras del texto de entrada
    palabras_eliminables = ['DATOS', 'Y', 'O', 'EN', 'PARA', 'CALIDAD', 'DE', 'PIE', 'CABECERA', 'MOSTRADOS']

    # Creamos a continuación una lista con palabras a partir de las cuales crear las sugerencias
    sugerencias = [
        'pulsar|Pulsar|clic|seleccionar|Seleccionar',
        'tooltip|tool-tip|hover',
        'aparece|contiene|Constitución|constitucion',
        ' habilitado|deshabilitado'
        ]

    i = 0
    while i < len(df_modelo):
        # Tanto las sugerencias como la búsqueda de posibles índices que contengan los datos se harán para aquellas filas donde el valor de la columna Tipo es Case
        if df_modelo.loc[i]['Tipo'] == 'Case' and df_modelo.loc[i]['YAML'] is not np.NaN:
            # Buscamos en primer lugar los posibles índices en que pueda encontrarse lo que nos interesa
            palabras = list(set(re.findall(r'\b[\w|0-9|\.|\\p{L}\\p{M}\\p{N}]+\b|".+"', df_modelo.loc[i]['Test'])))
            palabras = [re.sub(r'\(|\)', '', palabra) for palabra in palabras if palabra.upper() not in palabras_eliminables]

            indices_xpaths_encontrados = df_xpaths.loc[df_xpaths['PALABRAS CLAVE'].str.contains('|'.join(palabras))].index.astype('str').tolist()
            if len(indices_xpaths_encontrados) > 0:

                # En caso de que el número de índices encontrados sea mayor a 30, insertamos retorno de carro cada 30 índices para una mejora de la visualización del fichero final
                if len(indices_xpaths_encontrados) > 30:
                    indices_xpaths_encontrados = ','.join(indices_xpaths_encontrados).split(',')
                    j = 30
                    while j < len(indices_xpaths_encontrados):
                        indices_xpaths_encontrados.insert(j, '\r\n')

                        j = j + 30

                df_modelo.at[i, 'POSIBLES_INDICES_DATOS'] = ','.join(indices_xpaths_encontrados)

            # Buscamos ahora rellenar la columna de sugerencias que puedan facilitar la tarea. Para ello, crearemos una lista con tuplas como contenido
            sugerencias_test = []
            texto_test = df_modelo.loc[i]['Test']

            for sugerencia in sugerencias:
                resultado_busqueda = re.search(sugerencia, texto_test)
                if resultado_busqueda is not None:
                    match sugerencia:
                        case 'pulsar|Pulsar|clic|seleccionar|Seleccionar':
                            if 'botón' in texto_test or 'icono' in texto_test:
                                if 'refresco' in texto_test or 'refrescar' in texto_test or 'actualizar' in texto_test:
                                    sugerencias_test.append('  - funcion: hacer_clic\r\n    params:\r\n      - xpath: //*[name()="path"][contains(@d, "M19,8L15")]')

                                elif 'configuración' in texto_test:
                                    sugerencias_test.append('  - funcion: hacer_clic\r\n    params:\r\n      - xpath: //*[name()="path"][contains(@d, "M12,15.5A3")]')

                                elif 'añadir' in texto_test:
                                    sugerencias_test.append('  - funcion: hacer_clic\r\n    params:\r\n      - xpath: //*[name()="path"][contains(@d, "M19,13H13")]')

                                elif 'papelera' in texto_test or 'borrar' in texto_test:
                                    sugerencias_test.append('  - funcion: hacer_clic\r\n    params:\r\n      - xpath: //*[name()="path"][contains(@d, "M9,3V4H4")]')

                                else:
                                    resultado_busqueda = re.search(r'botón("\w+"|\(.+\))', texto_test)
                                    if resultado_busqueda is not None:
                                        resultado_busqueda = resultado_busqueda.group(0)
                                        resultado_busqueda = resultado_busqueda.replace('"', '').replace('()', '').replace(')', '')
                                        sugerencias_test.append('  - funcion: hacer_clic\r\n    params:\r\n      - xpath: //button//*[contains(text(), "' + resultado_busqueda + '")]')

                        case 'tooltip|tool-tip|hover':
                            if '• ' in texto_test:
                                texto_aniadible = '  - funcion: comprobar_contenido_hover\r\n    params:\r\n      - xpath: \r\n        esperar: \r\n          - xpath: \r\n        tipo_comprobacion: y\r\n        contenido: '
                                if '\r\n' in texto_test:
                                    lineas_texto = texto_test.split('\r\n')

                                else:
                                    lineas_texto = texto_test.split('\n')

                                j = 0
                                while j < len(lineas_texto):
                                    if '•' in lineas_texto[j] or '-' in lineas_texto[j]:
                                        texto_a_aniadir = lineas_texto[j].replace('•', '').replace('-', '')
                                        while texto_a_aniadir.startswith(' '):
                                            texto_a_aniadir = texto_a_aniadir[1:]

                                        texto_aniadible = texto_aniadible + '\r\n          - xpath: //*[contains(text(), "' + texto_a_aniadir + '")]'

                                        j = j + 1

                                    else:
                                        lineas_texto.remove(lineas_texto[j])

                                sugerencias_test.append(texto_aniadible)

                            elif 'mensaje:' in texto_test or 'texto:' in texto_test or 'descripción:' in texto_test:
                                resultado_busqueda = re.search(r'(?:mensaje\:|texto\:|descripción\:).+\.{0,1}$', texto_test)
                                if resultado_busqueda is not None:
                                    resultado_busqueda = resultado_busqueda.group(0).replace('mensaje: ', '').replace('texto: ', '').replace('descripción: ', '').replace('"', '').replace('.', '')
                                    sugerencias_test.append('  - funcion: comprobar_contenido_hover\r\n    params:\r\n      - xpath: \r\n        esperar: \r\n          - xpath: \r\n        tipo_comprobacion: y\r\n        contenido: \r\n          - xpath: //*[contains(text(), "' + resultado_busqueda + '")]')

                        case 'aparece|contiene|Constitución|constitucion':
                            texto_aniadible = '  - funcion: comprobar_contenido_elemento_2\r\n    params:\r\n      - xpath: \r\n        tipo_comprobacion: y\r\n        contenido: '
                            if 'texto "' in texto_test:
                                textos = re.findall('texto "\w+"', texto_test)
                                j = 0
                                while j < len(textos):
                                    texto_aniadible = texto_aniadible + '\r\n          - xpath: //*[contains(text(), "' + textos[j].replace('"', '') + '")]'
                                    j = j + 1

                                sugerencias_test.append(texto_aniadible)

                            if 'el campo' in texto_test or 'los campos' in texto_test:
                                textos = re.findall(r'[A-Z]{2,}[A-Z]{2,}|\".+\"', texto_test)
                                j = 0
                                while j < len(textos):
                                    texto_aniadible = texto_aniadible + '\r\n          - xpath: //*[contains(text(), "' + textos[j].replace('"', '') + '")]//following-sibling::input'
                                    j = j + 1

                                sugerencias_test.append(texto_aniadible)

                        case 'habilitado|habilita':
                            if 'botón' in texto_test:
                                resultado_busqueda = re.search(r'botón\s[A-Z]+.+(?:deshabilitado|\shabilitado)', texto_test)
                                if resultado_busqueda is not None:
                                    resultado_busqueda = resultado_busqueda.group(0)
                                    if ' habilitado' in resultado_busqueda:
                                        sugerencias_test.append('  - funcion: comprobar_elemento_habilitado\r\n    params:\r\n      - xpath: //button//*[contains(text(), "' + resultado_busqueda.split()[1] + '")]\r\n        habilitado: true')

                                    else:
                                        sugerencias_test.append('  - funcion: comprobar_elemento_habilitado\r\n    params:\r\n      - xpath: //button//*[contains(text(), "' + resultado_busqueda.split()[1] + '")]\r\n        habilitado: false')

                                else:
                                    resultado_busqueda = re.search(r'(?:"\w+"|\(.+\)).+(?:habilitado)', texto_test)
                                    if resultado_busqueda is not None:
                                        resultado_busqueda = resultado_busqueda.group(0)
                                        texto_boton = re.search(r'("\w+"|\(.+\))', texto_test)
                                        if texto_boton is not None:
                                            texto_boton = texto_boton.group(0)
                                            texto_boton = texto_boton.replace('"', '').replace('()','').replace(')', '')

                                            if ' habilitado' in resultado_busqueda:
                                                sugerencias_test.append('  - funcion: comprobar_elemento_habilitado\r\n    params:\r\n      - xpath: //button//*[contains(text(), "' + texto_boton + '")]\r\n        habilitado: true')

                                            else:
                                                sugerencias_test.append('  - funcion: comprobar_elemento_habilitado\r\n    params:\r\n      - xpath: //button//*[contains(text(), "' + texto_boton + '")]\r\n        habilitado: false')

            # Se eliminan aquellos elementos de la lista que no tengan contenido
            while '' in sugerencias_test:
                sugerencias_test.remove('')

            if len(sugerencias_test) > 0:
                df_modelo.at[i, 'SUGERENCIAS'] = '\r\n'.join(list(set(sugerencias_test)))

        i = i + 1

    # Guardamos como fichero XLSX con filtrado y preparado para su edición
    guardar_df_como_excel(df_modelo, directorio_ficheros_excel + '\\MODELO.xlsx', aplicar_colorizacion=False)

obtener_posiciones_datos_y_sugerencias()

Una vez completado el proceso, se deberá completar la columna de YAML, contenedora de las clases de las diferentes instancias. Para ello, el fichero generado a partir de la exploración de la página web y las columnas de sugerencias y posibles índices serán de ayuda en esta parte del proceso.

Una vez se complete el proceso, se procederá a la agrupación de casos de prueba en el test correspondiente, de manera que una vez se entrene el modelo sea posible realizar pruebas particulares o pruebas más globales. Si, por ejemplo, se tuviese un test denominado NTP y unos casos de prueba como comprobación de stratum y otro de comprobación de IP, se podría comprobar el stratum, la IP o ambos. A continuación puede verse el código utilizado para dicho propósito:

In [ ]:
def agrupar_casos_en_test():
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
    df_modelo = pd.read_excel(io=directorio_ficheros_excel + 'MODELO.xlsx', sheet_name='MODELO', index_col=None)

    # Obtenemos los títulos sin valores repetidos
    titulos = df_modelo['Titulo'].unique()
    for titulo in titulos:
        # Obtenemos los YAML de todos los casos para ese test
        yamls = df_modelo.loc[(df_modelo['Titulo'].str.contains(titulo)) & (df_modelo['Tipo'].str.contains('Case'))]['YAML']

        # Eliminamos aquellos que únicamente tengan el esqueleto
        yamls = [yaml for yaml in yamls if 'http' in yaml]

        # Obtenemos el índice del Test con dicho título
        indice_test = df_modelo.loc[(df_modelo['Titulo'].str.contains(titulo)) & (df_modelo['Tipo'].str.contains('Test'))]
        if len(indice_test) > 0:
            indice_test = indice_test.index.tolist()[0]

            # Insertamos para dicho test
            df_modelo.at[indice_test, 'YAML'] = '[\r\n' + ',\r\n\r\n'.join(yamls) + '\r\n]'

    # Guardamos el modelo con los cambios
    guardar_df_como_excel(df_modelo, directorio_ficheros_excel + 'MODELO.xlsx', aplicar_colorizacion=False)

agrupar_casos_en_test()

En la siguiente celda descargaremos los modelosHelsinki-NLP/opus-mt-es-en y Helsinki-NLP/opus-mt-en-es, que servirán para realizar Data Augmentation mediante la técnica de back translation

In [ ]:
snapshot_download(repo_id='Helsinki-NLP/opus-mt-es-en', cache_dir='./huggingface_mirror')
snapshot_download(repo_id='Helsinki-NLP/opus-mt-en-es', cache_dir='./huggingface_mirror')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')

Una vez se tiene un primer modelo, se han de incrementar el número de atributos para una misma clase o textos para una misma etiqueta, es decir, incrementar el número de textos que producen el mismo YAML de salida, para que así sea más posible que una vez entrenado el modelo se entiendan mejor las acciones que desea realizar el usuario. Para ello, se utilizarán técnicas de Data Augmentation con la librería nlpaug, diseñada para la realización de la tarea de forma sencilla.

In [5]:
def aumentar_datos_tests_nlpaug():
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
    
    # Se comprueba en primer lugar si se trata de retomar la tarea o de comenzar desde el principio, en cuyo caso existirá un fichero pickle guardado
    try:
        with open('df_modelo.pickle', mode='rb') as fichero_pickle:
            df_modelo = pickle.load(fichero_pickle)
            df_modelo_aumentado = pickle.load(fichero_pickle)
            i = pickle.load(fichero_pickle)
            
            # De alcanzarse este punto sin errores, es que se trata de retomar la tarea. Se inicia el contador
            print('Índice de inicio:', i)

    # Si se produce un error, se debe a que no existe el fichero. Lo capturamos y no hacemos nada
    except (FileNotFoundError, EOFError):
        print('No existe el fichero df_modelo.pickle o se ha producido un error. Se comienza desde el inicio.')
        
        # Se abre el fichero Excel que contiene el modelo base
        df_modelo = pd.read_excel(io=directorio_ficheros_excel + 'MODELO.xlsx', sheet_name='MODELO', index_col=None)
        i = 0
        
        # Creamos un dataframe auxiliar, de manera que el test original y aquellos generados con la aumentación queden consecutivos
        df_modelo_aumentado = pd.DataFrame(columns=df_modelo.columns)

    # Definimos el aumentador de sinónimos
    aumentador_sinonimos = aumentador_palabras.SynonymAug(aug_src='wordnet', lang='spa')

    # Definimos el aumentador mediante el método de back translation
    aumentador_back_translation = aumentador_palabras.back_translation.BackTranslationAug(from_model_name='Helsinki-NLP/opus-mt-es-en', to_model_name='Helsinki-NLP/opus-mt-en-es', name='BackTranslationAug', device='cpu', batch_size=32, max_length=512, force_reload=False, verbose=0)

    try:
        # Obtenemos en el siguiente bucle todas las filas, e iremos aplicando la técnica fila a fila
        barra_de_progreso = tqdm(unit='texto', total=len(df_modelo)-i)
        while i < len(df_modelo):
            # Guardamos una copia en el archivo df_modelo.pickle para, en caso de fallo, poder retomar a partir de un punto de guardado
            if i > 0 and i % 50 == 0:
                with open('df_modelo.pickle', mode='wb') as fichero_pickle:
                    pickle.dump(df_modelo, fichero_pickle)
                    pickle.dump(df_modelo_aumentado, fichero_pickle)
                    pickle.dump(i+1, fichero_pickle)
            
            # En primer lugar añadimos la fila existente al modelo final
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = df_modelo.loc[i]
    
            # Obtenemos el texto a partir del cual se generarán otros
            texto_test = df_modelo.loc[i]['Test']
    
            # Generamos los nuevos textos, primero mediante aumentación por sinónimos
            nuevo_texto = aumentador_sinonimos.augment(texto_test)
    
            # Añadimos los nuevos textos al modelo final
            nueva_fila = df_modelo.loc[i]
            nueva_fila['Test'] = nuevo_texto.replace('[', '').replace(']', '')
            
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila
            
            # Generamos nuevos textos mediante la ténica de back translation, generando 4 nuevos textos a partir del original
            nuevo_texto = aumentador_back_translation.augment(texto_test, n=4, num_thread=4)
    
            # Añadimos los nuevos textos al modelo final
            nueva_fila = df_modelo.loc[i]
            j = 0
            while j < len(nuevo_texto):
                nueva_fila['Test'] = nuevo_texto[j]
                
                df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila
                j = j + 1

            barra_de_progreso.update(1)
            i = i + 1

        print('Se ha finalizado la aumentación de datos. Guardando el modelo aumentado')
        
        # Guardamos el modelo con los cambios
        guardar_df_como_excel(df_modelo_aumentado, directorio_ficheros_excel + 'MODELO_AUMENTADO.xlsx', aplicar_colorizacion=False)

    except KeyboardInterrupt:
        print('Se ha detectado la ejecución de Ctrl+C. Guardando hasta lo realizado')
        with open('df_modelo.pickle', mode='wb') as fichero_pickle:
            pickle.dump(df_modelo, fichero_pickle)
            pickle.dump(df_modelo_aumentado, fichero_pickle)
            pickle.dump(i+1, fichero_pickle)

aumentar_datos_tests_nlpaug()

Índice de inicio: 2


100%|███████████████████████████████████████████████████████████████████████▊| 1037/1039 [14:24:26<01:33, 46.52s/texto]

Se ha finalizado la aumentación de datos. Guardando el modelo aumentado


100%|███████████████████████████████████████████████████████████████████████▊| 1037/1039 [14:24:27<01:40, 50.02s/texto]


In [7]:
directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
df_modelo_aumentado = pd.read_excel(io=directorio_ficheros_excel + 'MODELO_AUMENTADO.xlsx', sheet_name='MODELO_AUMENTADO', index_col=None)
df_modelo_aumentado_final = pd.DataFrame(columns=df_modelo_aumentado.columns)

tests = df_modelo_aumentado['Test'].tolist()
i = 0
while i < len(tests):
    if tests[i].startswith('['):
        lista_tests = tests[i].split("', '")
        fila_a_aniadir = df_modelo_aumentado.loc[i]
        for elemento in lista_tests:
            fila_a_aniadir['Test'] = elemento.replace('[', '').replace(']', '').replace('\'', '')
            df_modelo_aumentado_final.loc[len(df_modelo_aumentado_final)] = fila_a_aniadir

        l = 0

    else:
        df_modelo_aumentado_final.loc[len(df_modelo_aumentado_final)] = df_modelo_aumentado.loc[i]

    i = i + 1

guardar_df_como_excel(df_modelo_aumentado_final, directorio_ficheros_excel + 'MODELO_AUMENTADO.xlsx', aplicar_colorizacion=False)

La siguiente función busca comprobar el resultado de realizar Data Augmentation con ayuda de la librería Transformers, más concretamente con el modelo y tokenizador de MarianMT. Parte del código de la función se encuentra basado en el ejemplo disponible en https://huggingface.co/docs/transformers/model_doc/marian

In [9]:
def aumentar_datos_tests_marianmt():
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'

    # Se realiza en este punto la importación de modelos y tokenizadores de MarianMT para las traducciones en ambos sentidos
    modelo_es_en = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-es-en', max_position_embeddings=512)
    modelo_en_es = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-es', max_position_embeddings=512)
    tokenizador_marian_es_en = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-es-en', cache_dir='./huggingface_mirror', local_files_only=True, from_pt=True)
    tokenizador_marian_en_es = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-es', cache_dir='./huggingface_mirror', local_files_only=True, from_pt=True)
    
    # Se comprueba en primer lugar si se trata de retomar la tarea o de comenzar desde el principio, en cuyo caso existirá un fichero pickle guardado
    try:
        with open('df_modelo_marianmt.pickle', mode='rb') as fichero_pickle:
            df_modelo = pickle.load(fichero_pickle)
            df_modelo_aumentado = pickle.load(fichero_pickle)
            i = pickle.load(fichero_pickle)
            
            # De alcanzarse este punto sin errores, es que se trata de retomar la tarea. Se inicia el contador
            print('Índice de inicio:', i)

    # Si se produce un error, se debe a que no existe el fichero. Lo capturamos y no hacemos nada
    except (FileNotFoundError, EOFError):
        print('No existe el fichero df_modelo_marinamt.pickle o se ha producido un error. Se comienza desde el inicio.')
        
        # Se abre el fichero Excel que contiene el modelo base
        df_modelo = pd.read_excel(io=directorio_ficheros_excel + 'MODELO.xlsx', sheet_name='MODELO', index_col=None)
        i = 0
        
        # Creamos un dataframe auxiliar, de manera que el test original y aquellos generados con la aumentación queden consecutivos
        df_modelo_aumentado = pd.DataFrame(columns=df_modelo.columns)

    # Se procede al borrado de aquellas filas que contienen únicamente el esqueleto, si las hubiese, dado que dichos textos no formarán parte del modelo final
    df_modelo = df_modelo.loc[(df_modelo['YAML'] != np.NaN) &(df_modelo['YAML'].str.contains('http'))]

    # Se realiza un reseteo de índices para que no se produzcan errores en la obtención de filas
    df_modelo.reset_index(inplace=True, drop=True)

    # Se define a continuación un diccionario de sinónimos y qeuivalencias, ya sea una o varias palabras equivalentes entre sí para la aumentación de datos
    # El motivo de la generación de este diccionario se basa en la falta de contexto en el intento de aumentación por sinónimos de manera automática
    # El análisis de datos que contiene la cantidad de ocurrencias y la revisión manual del modelo inicial fueron de utilidad para la observación de las palabras y expresionesm más utilizadas
    diccionario_sinonimos = {
        'Comprobar': 'Verificar', 'comprobar': 'verificar',
        'Verificar': 'Comprobar', 'verificar': 'comprobar',
        'Configuración': 'Conformación', 'configuración': 'conformación',
        'área': 'sección', 'Área': 'Sección',
        'al pulsar': 'al hacer clic', 'pulsando': 'haciendo clic',
        'hacer hover': 'poner el ratón', 'haciendo hover': 'poniendo el ratón',
        'Constitución': 'Contenido', 'constitución': 'contenido',
        'Contenido': 'Constitución', 'contenido': 'constitución',
        'Campo': 'Columna/Elemento', 'campo': 'columna/elemento',
        'Columna': 'Campo', 'columna': 'campo',
        'Backup': 'Copia de seguridad', 'backup': 'copia de seguridad',
        'Máximo': 'Total', 'máximo': 'total',
        'Total': 'Máximo', 'total': 'máximo',
        'la leyenda mostrada': 'el texto mostrado', 'la calidad': 'la veracidad',
        'la veracidad': 'la calidad', 'un tooltip': 'una ventana emergente/un mensaje emergente',
        'un tool-tip': 'una ventana emergente/un mensaje emergente'
    }
    
    try:
        # Obtenemos la cantidad total de instancias o filas del modelo
        filas_modelo = len(df_modelo)

        # En esta sección se ejecutará la técnica de aumentación de sustitución de caracteres, que la librería easy-nlp-augmentation realiza directamente sobre el dataframe
        #clases_a_aumentar = df_modelo['YAML'].unique().tolist()
        #df_aumentado = augment_random_character(df_modelo, clases_a_aumentar, augmentation_percentage=1, text_column='Test')
        #print(df_modelo)
        #print(df_modelo_aumentado)
        #print(len(df_modelo))
        #print(len(df_aumentado))
        #input('parar')
        # Obtenemos en el siguiente bucle todas las filas, e iremos aplicando la técnica fila a fila
        barra_de_progreso = tqdm(unit='texto', total=filas_modelo-i, position=0)
        while i < filas_modelo:
            # Guardamos una copia en el archivo df_modelo_marianmt.pickle para, en caso de fallo, poder retomar a partir de un punto de guardado
            if i > 0 and i % 50 == 0:
                with open('df_modelo_marianmt.pickle', mode='wb') as fichero_pickle:
                    pickle.dump(df_modelo, fichero_pickle)
                    pickle.dump(df_modelo_aumentado, fichero_pickle)
                    pickle.dump(i+1, fichero_pickle)
            
            # En primer lugar añadimos la fila existente al modelo final
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = df_modelo.loc[i]
    
            # Obtenemos el texto a partir del cual se generarán otros
            texto_test = df_modelo.loc[i]['Test']

            # Se procede en este punto a la traducción, en primer lugar de español a inglés y posteriormente de ingés a español
            # De español castellano a inglés
            try:
                traduccion_es_en = modelo_es_en.generate(**tokenizador_marian_es_en(texto_test, return_tensors="pt", padding=True))
                texto_traduccion = [tokenizador_marian_es_en.decode(traduccion, skip_special_tokens=True) for traduccion in traduccion_es_en]

            except IndexError:
                print('Se ha detectado un texto con más de 512 tokens. Se procede a recortar desde el inicio hasta el máximo permitido.')
                # El error es debido a que supera el numero máximo de tokens (512). De ser así, se obtiene hasta el máximo, eliminando el resto
                texto_tokenizado = tokenizador_marian_es_en.tokenize(texto_test)
        
                texto_tokenizado = texto_tokenizado[:511]
                texto_test = ''.join(texto_tokenizado).replace('▁', ' ')
                traduccion_es_en = modelo_es_en.generate(**tokenizador_marian_es_en(texto_test, return_tensors="pt", padding=True))
                texto_traduccion = [tokenizador_marian_es_en.decode(traduccion, skip_special_tokens=True) for traduccion in traduccion_es_en]
                
            # De inglés a español castellano
            traduccion_en_es = modelo_en_es.generate(**tokenizador_marian_en_es(texto_traduccion, return_tensors="pt", padding=True))
            nuevo_texto_back_translation = [tokenizador_marian_en_es.decode(traduccion, skip_special_tokens=True) for traduccion in traduccion_en_es]

            # Se modifica el valor de la columna Test para el caso de prueba, introduciéndose el texto aumentado
            nueva_fila = df_modelo.loc[i]
            nueva_fila['Test'] = nuevo_texto_back_translation[0]
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila

            #palabras_expresiones_encontradas = [palabra_expresion for palabra_expresion in palabras_expresiones_texto if palabra_expresion in diccionario_sinonimos.keys()]
            claves_diccionario_sinonimos = list(diccionario_sinonimos.keys())

            # Se elimina toda palabra o expresión que no se encuentre en la lista de claves
            palabras_expresiones_encontradas = [clave for clave in claves_diccionario_sinonimos if clave in texto_test]
            
            # De haber más de 3 palabras o expresiones, se hace una selección aleatoria de entre 3 y la cantidad de palabras o expresiones encontradas
            if palabras_expresiones_encontradas is not None and len(palabras_expresiones_encontradas) > 3:
                total_lista = random.randint(3, len(palabras_expresiones_encontradas))
                random.shuffle(palabras_expresiones_encontradas)
                palabras_expresiones_encontradas = palabras_expresiones_encontradas[:total_lista]
                
            # Se sustitutyen dichas palabras y expresiones sobre el texto original
            nuevo_texto_sinonimos = texto_test
            for palabra_expresion in palabras_expresiones_encontradas:
                nuevo_texto_sinonimos = nuevo_texto_sinonimos.replace(palabra_expresion, diccionario_sinonimos[palabra_expresion])

            if nuevo_texto_sinonimos != texto_test:
                nueva_fila['Test'] = nuevo_texto_sinonimos
                df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila

            # Se procede a continuación a realizar data augmentation mediante reemplazo de caracteres. La cantidad de caracteres a reemplazar se determinará de forma aleatoria
            texto_test = df_modelo.loc[i]['Test']

            # Obtenemos un valor entero aleatorio para determinar el número de caracteres a modificar
            caracteres_a_modificar = random.randint(1,3)
            while caracteres_a_modificar > 0:
                # Se elige de forma aleatoria la posición a modificar
                posicion = random.randint(0,len(texto_test)-1)
                if texto_test[posicion] >= 'a' and texto_test[posicion] <= 'z':
                    minusculas = string.ascii_lowercase
                    minusculas = minusculas.replace(texto_test[posicion], '')
                    texto_test = texto_test[:posicion] + random.choice(minusculas) + texto_test[posicion+1:]
                    caracteres_a_modificar = caracteres_a_modificar - 1
                    
                elif texto_test[posicion] >= 'A' and texto_test[posicion] <= 'Z':
                    mayusculas = string.ascii_uppercase
                    mayusculas = mayusculas.replace(texto_test[posicion], '')
                    texto_test = texto_test[:posicion] + random.choice(mayusculas) + texto_test[posicion+1:]
                    caracteres_a_modificar = caracteres_a_modificar - 1
                    
            nueva_fila['Test'] = texto_test
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila
            
            # En este punto se generará un nuevo atributo a partir del existente, a partir del título del caso de prueba y del test
            # Se completará y corregirá según corresponda en la revisión manual
            texto_test = df_modelo.loc[i]['Test']
            tipo = df_modelo.loc[i]['Tipo']
            titulo = df_modelo.loc[i]['Titulo']

            # Se eliminan espacios que puedan existir al inicio o final
            while titulo.startswith(' '):
                titulo = titulo[1:]

            while titulo.endswith(' '):
                titulo = titulo[:-1]
            
            if tipo == 'Case':
                if re.fullmatch(r'^([A-Z]+|\s{0,1}|Á|É|Í|Ó|Ú)+$', texto_test) is not None:
                    nuevo_texto_test = 'Validar ' + texto_test.split('\n')[0].replace('\r', '') + ' en ' + titulo.lower()

                else:
                    nuevo_texto_test = 'Validar ' + texto_test.split('\n')[0].replace('\r', '') + ' en ' + titulo.replace(titulo[0], titulo[0].lower())

            elif tipo == 'Test':
                if re.fullmatch(r'^([A-Z]+|\s{0,1}|Á|É|Í|Ó|Ú)+$', texto_test) is not None:
                    nuevo_texto_test = 'Validar ' + titulo.lower()

                else:
                    nuevo_texto_test = 'Validar ' + titulo.replace(titulo[0], titulo[0].lower())

            # Se añade la nueva fila al modelo final
            nueva_fila['Test'] = nuevo_texto_test
            df_modelo_aumentado.loc[len(df_modelo_aumentado)] = nueva_fila
            
            barra_de_progreso.update(1)
            i = i + 1

        print('Se ha finalizado la aumentación de datos. Guardando el modelo aumentado')
        
        # Guardamos el modelo con los cambios
        guardar_df_como_excel(df_modelo_aumentado, directorio_ficheros_excel + 'MODELO_AUMENTADO_MARIANMT.xlsx', aplicar_colorizacion=False)

    except KeyboardInterrupt:
        print('Se ha detectado la ejecución de Ctrl+C. Guardando hasta lo realizado')
        with open('df_modelo_marianmt.pickle', mode='wb') as fichero_pickle:
            pickle.dump(df_modelo, fichero_pickle)
            pickle.dump(df_modelo_aumentado, fichero_pickle)
            pickle.dump(i+1, fichero_pickle)

aumentar_datos_tests_marianmt()

No existe el fichero df_modelo_marinamt.pickle o se ha producido un error. Se comienza desde el inicio.


 92%|███████████████████████████████████████████████████████████████████████      | 237/257 [22:42<02:19,  6.96s/texto]Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors


Se ha detectado un texto con más de 512 tokens. Se procede a recortar desde el inicio hasta el máximo permitido.


100%|█████████████████████████████████████████████████████████████████████████████| 257/257 [25:41<00:00,  9.29s/texto]

Se ha finalizado la aumentación de datos. Guardando el modelo aumentado


100%|█████████████████████████████████████████████████████████████████████████████| 257/257 [25:42<00:00,  6.00s/texto]


In [23]:
directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
df_modelo_aumentado = pd.read_excel(io=directorio_ficheros_excel + 'MODELO_AUMENTADO_MARIANMT.xlsx', sheet_name='MODELO_AUMENTADO_MARIANMT', index_col=None)
df_modelo_aumentado_final = pd.DataFrame(columns=df_modelo_aumentado.columns)

tests = df_modelo_aumentado['Test'].tolist()
i = 0
while i < len(tests):
    if tests[i].startswith('['):
        lista_tests = tests[i].split("', '")
        fila_a_aniadir = df_modelo_aumentado.loc[i]
        for elemento in lista_tests:
            fila_a_aniadir['Test'] = elemento.replace('[', '').replace(']', '').replace('\'', '')
            df_modelo_aumentado_final.loc[len(df_modelo_aumentado_final)] = fila_a_aniadir

        l = 0

    else:
        df_modelo_aumentado_final.loc[len(df_modelo_aumentado_final)] = df_modelo_aumentado.loc[i]

    i = i + 1

    # Eliminamos valores duplicados
    df_modelo_aumentado_final.drop_duplicates(inplace=True,ignore_index=True)

guardar_df_como_excel(df_modelo_aumentado_final, directorio_ficheros_excel + 'MODELO_AUMENTADO_MARIANMT.xlsx', aplicar_colorizacion=False)

Una vez completado el proceso, se procederá a una revisión del modelo resultante. En primer lugar se comprobará que para clase existen al menos 5 instancias, mostrándose un mensaje de no ser así. De tener todas las clases al menos 5 instancias, se procederá a guardar el modelo que se entrenará, eliminando las columnas que ya no son necesarias. Antes del guardado, se creará una nueva columna de etiquetas referente al tipo o tipo de acciones a realizar (hover o comprobar elementos de una ventana por ejemplo), que se obtendrá a partir del valor de funcion de la columna YAML:

In [33]:
def crear_etiquetas(yaml):
    # Se definen las acciones que no son relevantes
    acciones_descartables = ['hacer_clic', 'hacer_si', 'hacer_clic_derecho', 'ir_a_url', 'pulsar_tecla', 'buscar_elemento', 'buscar_elementos', 
                             'buscar_elementos_hijos', 'buscar_elemento_hijo', 'seleccionar_elemento', 'esperar_elemento', 'ir_a_centro',
                             'poner_quitar_pantalla_completa', 'obtener_fecha_y_hora', 'rellenar_campo', 'comprobar_url_actual', 'mover_elemento',
                             'iniciar_sesion', 'cerrar_sesion', 'hacer_hover', 'hacer_si_no_existe', 'guardar_valor', 'obtener_valor']

    # Se hace una primera separación por líneas
    acciones = yaml.split('\n') 
    acciones = [accion.replace('\r', '') for accion in acciones]
    
    # Se obtienen las acciones
    acciones = [accion.split('funcion:')[1] for accion in acciones if 'funcion' in accion]

    # Se eliminan los posibles espaciones que pueda contener al inicio del elemento
    i = 0
    while i < len(acciones):
        while acciones[i].startswith(' '):
            acciones[i] = acciones[i][1:]
            
        i = i + 1

    # Se eliminan aquellas que se han considerado descartables
    acciones = [accion for accion in acciones if accion not in acciones_descartables]

    # Se eliminan valores duplicados
    acciones = list(set(acciones))
    
    # Se devuelven las acciones. En caso de que la lista contenga más de un elemento, en primer lugar se ordenan sus elementos
    if len(acciones) == 1:
        return acciones[0]

    elif len(acciones) > 1:
        acciones.sort()
        return ','.join(acciones)

    else:
        return ''

def guardar_modelo_entrenamiento():
    directorio_ficheros_excel = os.getcwd() + '\\ficheros_excel\\'
    df_modelo = pd.read_excel(io=directorio_ficheros_excel + 'MODELO_AUMENTADO_MARIANMT.xlsx', sheet_name='MODELO_AUMENTADO_MARIANMT', index_col=None)

    # Se comprueba si todas las clases contienen 5 instancias
    clases = df_modelo['YAML'].unique()

    indices = []
    for clase in clases:
        filas = df_modelo.loc[df_modelo['YAML'] == clase]
        if len(filas) < 5:
            indices.append(filas.index[0] + 2)

    if len(indices) > 0:
        print('Se han de revisar las clases de las siguientes filas:\n', indices)
    
    else:
        # Se procede a eliminar las columnas que originalmente fueron usadas para la edición y generación del modelo y ya no resultan necesarias
        df_modelo.drop(columns=['Tipo', 'Titulo', 'SUGERENCIAS', 'POSIBLES_INDICES_DATOS'], inplace=True)

        df_modelo['acciones'] = df_modelo['YAML'].apply(crear_etiquetas)
        guardar_df_como_excel(df_modelo, directorio_ficheros_excel + 'MODELO_ENTRENAMIENTO.xlsx', aplicar_colorizacion=False)
        print('Se ha guardado el modelo de entrenamiento en ' + directorio_ficheros_excel + 'MODELO_ENTRENAMIENTO.xlsx')

guardar_modelo_entrenamiento()

Se ha guardado el modelo de entrenamiento en C:\Users\jmarrieta\PycharmProjects\TFM_NLP\ficheros_excel\MODELO_ENTRENAMIENTO.xlsx


En este punto el dataset se encuentra listo para el entrenamiento del modelo. Puede visuarlizarse en **entrenamiento_y_uso_modelo.ipynb**